In [1]:
import tensorflow_datasets as tfds

In [2]:
datasets = tfds.load(name="mnist")

Dataset mnist downloaded and prepared to /Users/jin/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
